# Trabalho de Recuperação de AED

**Professor** 
: Dr. Jefferson Oliveira Andrade

**Aluno**
: <font color="red">Insira Seu Nome Aqui</font>

-----

# Introdução

Este trabalho de recuperação consistem em fazer um pequeno projeto de ciência de dados. Você deve usar as técnicas que vimos ao longo desta disciplina para a análise exploratória de dados e a criação de modelos simples.

O trabalho valerá 100 pontos.

A nota final da disciplina será calculada por média aritmética simples entre a nota da Etapa 1 e a nota da Recuperação.

# Definição do Problema

Os cientistas de dados das Lojas VendeMuito coletaram dados de vendas de 2013 para 1559 produtos em 10 lojas em diferentes cidades. Além disso, determinados atributos de cada produto e loja foram definidos. O objetivo é criar um modelo preditivo e descobrir as vendas de cada produto em uma determinada loja.

Usando este modelo, as Lojas VendeMuito tentará entender as propriedades de produtos e lojas que desempenham um papel fundamental no aumento das vendas.

Observe que os dados podem ter valores faltantes, pois algumas lojas podem não relatar todos os dados devido a falhas técnicas. Portanto, será necessário tratá-los de acordo.


# Dados

Temos um conjunto de dados de treinamento (8523 linhas) e um conjunto de dados de teste (5681 linhas), o conjunto de dados de treinamento tem variável(s) de entrada e saída. Você precisa criar um modelo, usando os de treinamento, para prever as vendas para o conjunto de dados de teste.


| Variável                    | Descrição |
|----------------------------:|------------------|
| `Item_Identifier`           | ID exclusivo do produto |
| `Item_Weight`               | Peso do produto |
| `Item_Fat_Content`          | Se o produto é baixo teor de gordura ou não |
| `Item_Visibility`           | O percentual  da área total de exibição de todos os produtos em uma loja alocada para o produto específico |
| `Item_Type`                 | A categoria à qual o produto pertence |
| `Item_MRP`                  | Preço Máximo de Varejo (preço de tabela) do produto |
| `Outlet_Identifier`         | ID da loja exclusiva |
| `Outlet_Establishment_Year` | O ano em que a loja foi estabelecida |
| `Outlet_Size`               | O tamanho da loja em termos de área coberta |
| `Outlet_Location_Type`      | O tipo de cidade em que a loja está localizada |
| `Outlet_Type`               | Se a tomada é apenas uma mercearia ou algum tipo de supermercado |
| `Item_Outlet_Sales`         | Vendas do produto na loja particular |

A variável de resultado, i.e., a que deve ser prevista é a variável `Item_Outlet_Sales`.

Os arquivos de dados estão disponíveis para download na página desta atividade no AVA. Os arquivos de treinamento e de teste se chamam, respectivamente, `treina-vendemuito.csv` e `testa-vendemuito.csv`. 


# Métrica de Avaliação

O desempenho do seu modelo será avaliado com base na sua previsão das vendas para os dados de teste. Seu notebook deverá gerar um arquivo chamado `test-predict.csv` contendo três campos: `Item_Identifier`, `Outlet_Identifier`, `Item_Outlet_Sales`. A avaliação destes resultados será feita por um *script*, então não mude o nome do arquivo ou as colunas que ele contém. O arquivo CSV gerado deve conter o cabeçalho com o nome das colunas.

Suas previsões serão comparadas com os dados de “ground truth” e serão avaliadas com o quão próximas elas ficaram destes dados. Nós usaremos o valor *Root Mean Square Error* (RMSE) para julgar sua resposta.

\begin{equation}
\mathop{RMSE} = \sqrt{\frac{\sum_{i=1}^N\left(\mathrm{predito}_i - \mathrm{real}_i\right)^2}{N}}
\end{equation}

O seu notebook **não** deve conter o cálculo do RMSE. Esse cálculo será executado pelo *script* que irá avaliar o seu resultado.

Note que o RMSE será apenas um dos componentes da nota. Também serão avaliados a clareza e organização do trabalho apresentado, os métodos de pré-processamento de dados utilizados, os métodos de visualização de dados usados na análise exploratória, as justificativas para as escolhas de quais dimensões usar no modelo, e a justificativa para a escolha de um tipo de modelo específico.

Para esta atividade, você deve entrega apenas o arquivo de notebook com a sua solução. Seu notebook será executado pelo professor para gerar o arquivo com os resultados. Portanto, assegure-se de que ele não esteja com erros de execução.

Esta atividade deve ser entregue, **impreterivelmente**, até as 21h do dia 21 de dezembro de 2018.


-------
-------

# Resolução

-------
-------

In [1]:
# "de cada produto em uma determinada loja"

In [43]:
# import required libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import seaborn as sns; sns.set()
# from collections import defaultdict
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import PolynomialFeatures
# from sklearn.utils import resample

In [44]:
# set gaussian features class
class GaussianFeatures(BaseEstimator, TransformerMixin):
    """Uniformly spaced Gaussian features for one-dimensional input"""
    
    def __init__(self, N, width_factor=2.0):
        self.N = N
        self.width_factor = width_factor
        
    @staticmethod
    def _gauss_basis(x, y, width, axis=None):
        arg = (x - y) / width
        return np.exp(-0.5 * np.sum(arg ** 2, axis))
    
    def fit(self, X, y=None):
        # create N centers spread along the data range
        self.centers_ = np.linspace(X.min(), X.max(), self.N)
        self.width_ = self.width_factor * (self.centers_[1] - self.centers_[0])
        return self
    
    def transform(self, X):
        return self._gauss_basis(X[:, :, np.newaxis], self.centers_, self.width_, axis=1)

In [46]:
# set lib parameters
%matplotlib inline
np.random.seed(1)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [36]:
# load

test = pd.read_csv('testa-vendemuito.csv', index_col='Item_Identifier').astype(float, errors='ignore')
train = test = pd.read_csv('treina-vendemuito.csv', index_col='Item_Identifier').astype(float, errors='ignore')

train.isna().sum()

Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [37]:
# cleanup train dataset
# not optimized but works!

df = train
dict_item_weight = {}
dict_outlet_size = {}
item_mean = train['Item_Weight'].mean()

df['Item_Id'] = df.index

for line in train[train['Item_Weight'].notna()].values:
    item_id = line[11]
    item_weight = line[0]
    dict_item_weight[item_id] = item_weight

for line in train[train['Item_Weight'].isna()].values:
    item_id = line[11]
    if item_id in dict_item_weight:
        item_weight = dict_item_weight[item_id]
    else: item_weight = item_mean
    train.at[item_id, 'Item_Weight'] = float(item_weight)

for line in train[train['Outlet_Size'].notna()].values:
    outlet_id = line[5]
    outlet_size = line[7]
    dict_outlet_size[outlet_id] = outlet_size

for line in train[train['Outlet_Size'].isna()].values:
    item_id = line[11]
    outlet_id = line[5]
    if outlet_id in dict_outlet_size:
        outlet_size = dict_outlet_size[outlet_id]
    else: outlet_size = 'Medium' # <-- most frequent
    train.at[item_id, 'Outlet_Size'] = outlet_size
    
train = df
train.isna().sum()

Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
Item_Id                      0
dtype: int64

In [38]:
# cleanup test dataset
# not optimized but works!

df = test
dict_item_weight = {}
dict_outlet_size = {}
item_mean = test['Item_Weight'].mean()

df['Item_Id'] = df.index

for line in test[test['Item_Weight'].notna()].values:
    item_id = line[11]
    item_weight = line[0]
    dict_item_weight[item_id] = item_weight

for line in test[test['Item_Weight'].isna()].values:
    item_id = line[11]
    if item_id in dict_item_weight:
        item_weight = dict_item_weight[item_id]
    else: item_weight = item_mean
    test.at[item_id, 'Item_Weight'] = float(item_weight)

for line in test[test['Outlet_Size'].notna()].values:
    outlet_id = line[5]
    outlet_size = line[7]
    dict_outlet_size[outlet_id] = outlet_size

for line in test[test['Outlet_Size'].isna()].values:
    item_id = line[11]
    outlet_id = line[5]
    if outlet_id in dict_outlet_size:
        outlet_size = dict_outlet_size[outlet_id]
    else: outlet_size = 'Medium' # <-- most frequent
    test.at[item_id, 'Outlet_Size'] = outlet_size
    
test = df
test.isna().sum()

Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
Item_Id                      0
dtype: int64

In [66]:
# regression analysis
X = train.values
y = train['Item_Outlet_Sales']

# model = LinearRegression(fit_intercept=False)
# model = make_pipeline(PolynomialFeatures(3), LinearRegression())
# model = make_pipeline(GaussianFeatures(20), LinearRegression())
# model = make_pipeline(GaussianFeatures(15), Ridge(alpha=0.01, max_iter=100))
model = make_pipeline(GaussianFeatures(30), Lasso(alpha=0.001, max_iter=1000))

model.fit(X, y)
train['Predicted_Outlet_Sales'] = model.predict(X)
train[['Item_Outlet_Sales', 'Predicted_Outlet_Sales']].plot(alpha=0.7)

TypeError: '<=' not supported between instances of 'float' and 'str'